In [27]:
import xmlrpc.client
import pymssql

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'  
database_name = 'Odoo_sql_database'   
sql_user_name = 'SuperAdmin'   
sql_password = 'SuperAdmin'

# Define the date range filter (1/1/2025 to 31/1/2025)
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2025-03-10')
]

# Authenticate and get user ID
common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common')
uid = common.authenticate(db_name, username, password, {})

if not uid:
    print("Failed to authenticate. Check your credentials.")
    exit()

# Connect to Odoo models
models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object')

# Connect to SQL Server
try:
    conn = pymssql.connect(server=server_name, user=sql_user_name, password=sql_password, database=database_name)
    cursor = conn.cursor()
    print("Connected to SQL Server successfully.")
except pymssql.Error as e:
    print(f"Error connecting to SQL Server: {e}")
    exit()

# Check if the table exists, and create it if it doesn't
table_name = 'order_details'
check_table_query = f"""
IF NOT EXISTS (
    SELECT * FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_NAME = '{table_name}'
)
BEGIN
    CREATE TABLE {table_name} (
        id INT IDENTITY(1,1) PRIMARY KEY,
        order_ref NVARCHAR(255),
        order_date DATETIME,
        unit_price DECIMAL(18, 2),
        quantity DECIMAL(18, 2),
        product_name NVARCHAR(255),
        product_cost DECIMAL(18, 2)
    );
END
"""
cursor.execute(check_table_query)
conn.commit()
print(f"Table '{table_name}' created or verified.")

# Fetch POS orders within the date range
print("Fetching POS orders...")
pos_orders = models.execute_kw(
    db_name, uid, password,
    'pos.order', 'search_read',
    [date_domain],
    {'fields': ['name', 'date_order', 'lines']}
)

# Process each order and extract required data
print("Extracting order details...")
for order in pos_orders:
    order_ref = order.get('name')
    order_date = order.get('date_order')
    order_lines = order.get('lines')

    #print(f"\nOrder Ref: {order_ref}")
    #print(f"Order Date: {order_date}")

    # Fetch details for each order line
    for line_id in order_lines:
        line_data = models.execute_kw(
            db_name, uid, password,
            'pos.order.line', 'read',
            [line_id],
            {'fields': ['price_unit', 'qty', 'product_id']}
        )

        if line_data:
            line = line_data[0]
            unit_price = line.get('price_unit')
            quantity = line.get('qty')
            product_id = line.get('product_id')[0] if line.get('product_id') else None

            # Fetch product details
            if product_id:
                product_data = models.execute_kw(
                    db_name, uid, password,
                    'product.product', 'read',
                    [product_id],
                    {'fields': ['display_name', 'standard_price']}
                )

                if product_data:
                    product = product_data[0]
                    product_name = product.get('display_name')
                    product_cost = product.get('standard_price')

                    # Print extracted data
                    #print(f"  Order Line:")
                    #print(f"    Unit Price: {unit_price}")
                    #print(f"    Quantity: {quantity}")
                    #print(f"    Product Name: {product_name}")
                    #print(f"    Product Cost: {product_cost}")

                    # Insert data into SQL Server
                    insert_query = f"""
                    INSERT INTO {table_name} (
                        order_ref, order_date, unit_price, quantity, product_name, product_cost
                    ) VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    cursor.execute(insert_query, (
                        order_ref, order_date, unit_price, quantity, product_name, product_cost
                    ))
                    conn.commit()
                    #print("    Data inserted into SQL Server.")

# Close the SQL Server connection
cursor.close()
conn.close()
print("SQL Server connection closed.")

Connected to SQL Server successfully.
Table 'order_details' created or verified.
Fetching POS orders...
Extracting order details...


KeyboardInterrupt: 

In [31]:
import xmlrpc.client
import pymssql
import time

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter (1/1/2025 to 31/1/2025)
date_domain = [
    ('date_order', '>=', '2025-01-01'),
    ('date_order', '<=', '2025-01-31')
]

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', timeout=300)
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', timeout=300)
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch orders
    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['name', 'date_order', 'lines'], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    # Step 3: Prefetch all order lines and products
    print("Fetching related order lines and products...")
    all_line_ids = [line_id for order in orders for line_id in order.get('lines', [])]
    order_lines = fetch_data_in_bulk('pos.order.line', ['id', 'price_unit', 'qty', 'product_id'], [['id', 'in', all_line_ids]], uid)

    all_product_ids = list({line.get('product_id', [])[0] for line in order_lines if line.get('product_id')})
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price'], [['id', 'in', all_product_ids]], uid)

    # Step 4: Map products for faster lookups
    product_map = {product['id']: product for product in products}

    # Step 5: Prepare and batch insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'order_details'
        )
        BEGIN
            CREATE TABLE order_details (
                id INT IDENTITY(1,1) PRIMARY KEY,
                order_ref NVARCHAR(255),
                order_date DATETIME,
                unit_price DECIMAL(18, 2),
                quantity DECIMAL(18, 2),
                product_name NVARCHAR(255),
                product_cost DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            order_date = order.get('date_order')
            line_ids = order.get('lines', [])

            for line_id in line_ids:
                line = next((l for l in order_lines if l['id'] == line_id), None)
                if not line:
                    continue

                unit_price = line.get('price_unit')
                quantity = line.get('qty')
                product_id = line.get('product_id', [])[0] if line.get('product_id') else None

                product_name, product_cost = None, None
                if product_id and product_id in product_map:
                    product = product_map[product_id]
                    product_name = product.get('display_name')
                    product_cost = product.get('standard_price')

                # Add to batch
                batch_data.append((order_ref, order_date, unit_price, quantity, product_name, product_cost))

                # Insert in batches
                if len(batch_data) >= batch_size:
                    cursor.executemany(f"""
                    INSERT INTO order_details (
                        order_ref, order_date, unit_price, quantity, product_name, product_cost
                    ) VALUES (%s, %s, %s, %s, %s, %s)
                    """, batch_data)
                    conn.commit()
                    batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO order_details (
                order_ref, order_date, unit_price, quantity, product_name, product_cost
            ) VALUES (%s, %s, %s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting data synchronization...
Authentication error: ServerProxy.__init__() got an unexpected keyword argument 'timeout'
